In [84]:
# CTD Calibration Parser for xmlcon file
import sys, os, csv
import datetime
import time
import xml.etree.ElementTree as et
from zipfile import ZipFile

class CTDCalibration(object):
    # Class that stores calibration values for CTDs.
    # \param self
    def __init__(self):
        self.coefficient_name_map = {
            'TA0': 'CC_a0',
            'TA1': 'CC_a1',
            'TA2': 'CC_a2',
            'TA3': 'CC_a3',
            'CPCOR': 'CC_cpcor',
            'CTCOR': 'CC_ctcor',
            'CG': 'CC_g',
            'CH': 'CC_h',
            'CI': 'CC_i',
            'CJ': 'CC_j',
            'G': 'CC_g',
            'H': 'CC_h',
            'I': 'CC_i',
            'J': 'CC_j',
            'PA0': 'CC_pa0',
            'PA1': 'CC_pa1',
            'PA2': 'CC_pa2',
            'PTEMPA0': 'CC_ptempa0',
            'PTEMPA1': 'CC_ptempa1',
            'PTEMPA2': 'CC_ptempa2',
            'PTCA0': 'CC_ptca0',
            'PTCA1': 'CC_ptca1',
            'PTCA2': 'CC_ptca2',
            'PTCB0': 'CC_ptcb0',
            'PTCB1': 'CC_ptcb1',
            'PTCB2': 'CC_ptcb2',
            # additional types for series O
            'C1': 'CC_C1',
            'C2': 'CC_C2',
            'C3': 'CC_C3',
            'D1': 'CC_D1',
            'D2': 'CC_D2',
            'T1': 'CC_T1',
            'T2': 'CC_T2',
            'T3': 'CC_T3',
            'T4': 'CC_T4',
            'T5': 'CC_T5',
        }

        self.o2_coefficients_map = {
            'A': 'CC_residual_temperature_correction_factor_a',
            'B': 'CC_residual_temperature_correction_factor_b',
            'C': 'CC_residual_temperature_correction_factor_c',
            'E': 'CC_residual_temperature_correction_factor_e',
            'SOC': 'CC_oxygen_signal_slope',
            'OFFSET': 'CC_frequency_offset'
        }
        
        # Initialize dictionary with calibration coefficient names and values
        self.coefficients = {}
        self.asset_tracking_number = None
        self.serial = None
        self.date = None
        self.type = 'CTD'
        
    def load_cal(self, filepath):
        """
        Function to read in calibration values for the CTD
        from the vendor supplied documentation. May be loaded
        directly from a zip directory w/o needing extraction.
        """
        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                if any('.cal' in x for x in zfile.namelist()):
                    findex, *ignore = [(i,x) for i,x in enumerate(zfile.namelist()) if '.cal' in x][0]
                    with zfile.open(zfile.namelist()[findex]) as file:
                        self.read_cal(file)
                elif any('.xmlcom' in x for x in zfile.namelist()):
                    findex, *ignore = [(i,x) for i,x in enumerate(zfile.namelist()) if '.xmlcon' in x][0]
                    with zfile.open(zfile.namelist()[findex]) as file:
                        self.read_xml(file)
                else:
                    raise Exception('No .cal or .xmlcon calibration file found.')
        elif filepath.endswith('.cal'):
            with open(filepath) as file:
                self.read_cal(file)
        elif filepath.endswith('.xmlcon'):
            with open(filepath) as file:
                self.read_xml(file)
        else:
            raise Exception('No .cal or .xmlcon calibration file found.')
            
                    
    def read_xml(self, calfile):
        """
        Function which parses an xmlcon file, loads the appropriate calibration
        coefficients, and saves them as attributes to the CTD object
        """
        
        tree = et.parse(calfile)
        root = tree.getroot()
        coeffs = {}
        
        Tflag = False
        O2flag = False

        for child in tree.iter():
            key = child.tag.upper()
            value = child.text.upper()
        
            # Do a couple of checks for type of CTD and flag for presence of
            # Oxygen sensor, Type (16+ vs 37)
            if key == 'NAME':
                if 'SEACAT' in value:
                    S16flag = True
                else:
                    S16flag = False
                
            if key == 'OXYGENSENSOR':
                O2flag = True
            elif 'SENSOR' in key and O2flag == True:
                O2flag = False
            else:
                pass
        
            if key == 'SERIALNUMBER':
                if self.serial is None and value is not None:
                    if S16flag == True:
                        self.serial = '16-' + value
                    else:
                        self.serial = value
        
            if key == 'CALIBRATIONDATE':
                if self.date is None and value is not None:
                    self.date = datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')
            
            # Have to rename the temperature keys to 'T'+key because fuck it, nothing is straightforward
            if key == 'TEMPERATURESENSOR':
                Tflag = True
            elif 'SENSOR' in key and Tflag == True:
                Tflag = False
            else:
                pass
        
            if Tflag == True:
                key = 'T'+key
                
            # Load the calibration into the CTD structure
            name = self.coefficient_name_map.get(key)
            if not name or name is None:
                pass
            else:
                self.coefficients[name] = value
                
            # If there is an oxygen sensor on the CTD, add the oxygen calibration
            # to the coefficient 
            if O2flag == True:
                o2_name = self.o2_coefficients_map.get(key)
                if not o2_name or o2_name is None:
                    pass
                else:
                    self.coefficients[o2_name] = value
                    
    
    def read_cal(self, file):
        """
        Function which opens and reads a .cal file. 
        """
        for line in file:
            data = line.decode('ASCII').split('=')
            key = data[0].strip()
            value = data[1].strip()
    
            if key == 'INSTRUMENT_TYPE' and value == 'SEACATPLUS':
                self.serial = '16-'
        
            if key == 'SERIALNO':
                self.serial = serial + value
        
            if key == 'CCALDATE':
                self.date = datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')
        
            name = self.coefficient_name_map.get(key)
            if not name or name is None:
                continue
            else:
                self.coefficients[name] = value
                
                
    def write_cal_csv(self, filepath):
        
        

In [85]:
filepath = "C:/Users/areed/Documents/Project_Files/Records/Instrument_Records/Sea-Bird/CTDBP-O_SBE_16PlusV2_SN_7249_Calibration_Files_2012-1-15.zip"

In [86]:
CTD = CTDCalibration()

In [87]:
CTD.coefficients

{}

In [88]:
CTD.load_cal(filepath)

In [89]:
CTD.coefficients

{'CC_a0': '1.239929e-003',
 'CC_a1': '2.769904e-004',
 'CC_a2': '-1.321497e-006',
 'CC_a3': '1.885808e-007',
 'CC_g': '-9.813059e-001',
 'CC_h': '1.527103e-001',
 'CC_i': '-4.202770e-004',
 'CC_j': '5.446040e-005',
 'CC_ctcor': '3.250000e-006',
 'CC_cpcor': '-9.570000e-008',
 'CC_C1': '-4.643779e+003',
 'CC_C2': '-7.540100e-002',
 'CC_C3': '8.852730e-004',
 'CC_D1': '6.781200e-002',
 'CC_D2': '0.000000e+000',
 'CC_T1': '3.005444e+001',
 'CC_T2': '-3.892070e-004',
 'CC_T3': '2.670250e-006',
 'CC_T4': '1.718780e-009',
 'CC_T5': '0.000000e+000'}